In [1]:

import sys
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from pl_bolts.datamodules import CIFAR10DataModule
import torchvision
from pl_bolts.transforms.dataset_normalizations import cifar10_normalization
from pytorch_lightning import LightningModule, seed_everything, Trainer
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
from torch.optim.lr_scheduler import OneCycleLR
from typing import Optional, Dict, List
from torch.optim.swa_utils import AveragedModel, update_bn
from torchmetrics.functional import accuracy
import argparse



In [5]:

def create_data_module(path, batch_size):
    train_transforms = torchvision.transforms.Compose([
        torchvision.transforms.RandomCrop(32, padding=4),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.ToTensor(),
        cifar10_normalization(),
    ])

    test_transforms = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        cifar10_normalization(),
    ])

    return CIFAR10DataModule(
        data_dir=path,
        batch_size=batch_size,
        num_workers=2,
        train_transforms=train_transforms,
        test_transforms=test_transforms,
        val_transforms=test_transforms,
    )


In [30]:


cifar10_dm = create_data_module("/tmp/aivanou", 32)
cifar10_dm.prepare_data()
cifar10_dm.setup("fit")

train_dl = cifar10_dm.train_dataloader()


Files already downloaded and verified
Files already downloaded and verified


In [33]:

batch = next(iter(train_dl))


In [35]:

x,y = batch


In [64]:

def get_dummy_model():
    conv1 = nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    relu = nn.ReLU()
    max_pool = nn.MaxPool2d(kernel_size=6, stride=2, padding=1)
    conv2 = nn.Conv2d(64, 32, kernel_size=(3, 3), stride=(3, 3), padding=(1, 1))
    relu = nn.ReLU()
    nn.AdaptiveAvgPool2d(output_size=(1,1))
    torch.flatten(out4, 1)
    nn.Linear(128, 100)
    nn.Linear(100, 10)
                         
    


<function torch.nn.functional.relu(input: torch.Tensor, inplace: bool = False) -> torch.Tensor>

In [65]:

out1 = F.relu(conv_layer(x))
out1.shape

torch.Size([32, 64, 32, 32])

In [66]:

pool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
out2 = pool1(out1)
out2.shape

torch.Size([32, 64, 16, 16])

In [107]:

conv2 = nn.Conv2d(64, 32, kernel_size=(3, 3), stride=(3, 3), padding=(1, 1))
out3=F.relu(conv2(out2))
out3.shape


torch.Size([32, 32, 6, 6])

In [117]:

conv3 = nn.Conv2d(32, 512, kernel_size=(6, 6), stride=(1, 1), padding=(1, 1))

conv3(out3).shape


torch.Size([32, 512, 3, 3])

In [108]:

pool2 = nn.MaxPool2d(kernel_size=6, stride=2, padding=1)
out4 = pool2(out3)
out4.shape



torch.Size([32, 32, 2, 2])

In [120]:

out5 = torch.flatten(nn.AdaptiveAvgPool2d(output_size=(1,1))(out4), 1)
out5.shape


torch.Size([32, 32])

In [122]:

out5 = torch.flatten(out4, 1)
out5.shape


torch.Size([32, 128])

In [123]:

fc1 = nn.Linear(128, 100)
out6 = F.relu(fc1(out5))
out6.shape


torch.Size([32, 100])

In [95]:

model = torchvision.models.resnet18(pretrained=False, num_classes=10)
# model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [83]:

??nn.Linear